# Import the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load movies and ratings datasets

In [ ]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

# User-Item Matrix for Collabrative Filtering

In [ ]:
user_item_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)

# Genre Matrix for Content-Based Filtering

In [ ]:
# Spliting genres into dummy variables
movies['genres'] = movies['genres'].fillna('')  # Ensure no missing genres; Fills NaN with empty string
genre_matrix = movies['genres'].str.get_dummies(sep='|')  # One-hot encode genres
genre_matrix.index = movies['movieId']  # Use movieId as index

# Compute the similarities using cosine similarity

In [ ]:
# user-user cosine similarity based on ratings
user_similarity = cosine_similarity(user_item_matrix.values)
user_similarity_df = pd.DataFrame(
    user_similarity, 
    index=user_item_matrix.index, 
    columns=user_item_matrix.index)

# item-item cosine similarity based on genres
item_similarity = cosine_similarity(genre_matrix.values)
item_similarity_df = pd.DataFrame(
    item_similarity, 
    index=genre_matrix.index, 
    columns=genre_matrix.index)

# Define a function to get collaborative filtering scores

In [ ]:
def get_collab_fil_scores(user_id, user_similarity_df, user_item_matrix):

    # Get the user's ratings
    user_ratings = user_item_matrix.loc[user_id]
    
    # Get similarities to other users
    user_similarities = user_similarity_df.loc[user_id]
    
    # Predict ratings for items the user hasn't rated
    cf_scores = {}
    for movie_id in user_item_matrix.columns:
        if user_ratings[movie_id] == 0:
            # Weighted average of other users' ratings
            weighted_sum = (user_item_matrix[movie_id] * user_similarities).sum()
            similarity_sum = user_similarities.sum()
            cf_scores[movie_id] = weighted_sum / similarity_sum if similarity_sum > 0 else 0
    
    return pd.Series(cf_scores).sort_values(ascending=False)

# Define a function to get content-based filtering scores

In [ ]:
def get_cont_fil_scores(user_id, user_item_matrix, item_similarity_df, threshold=3.0):

    user_ratings = user_item_matrix.loc[user_id]
    liked_movies = user_ratings[user_ratings >= threshold].index  # Movies rated above threshold

    cbf_scores = {}
    for movie_id in user_item_matrix.columns:
        if user_ratings[movie_id] == 0:
            similarities = item_similarity_df.loc[movie_id, liked_movies]
            cbf_scores[movie_id] = similarities.mean() if not similarities.empty else 0
    
    return pd.Series(cbf_scores).sort_values(ascending=False)

# Define a function to get hybrid filtering scores

In [ ]:
def get_hybrid_recommendations(user_id, w_cf=0.6, w_cbf=0.4, top_n=5):

    # Get scores from both methods
    cf_scores = get_collab_fil_scores(user_id, user_similarity_df, user_item_matrix)
    cbf_scores = get_cont_fil_scores(user_id, user_item_matrix, item_similarity_df)
    
    # Combine scores
    all_movies = set(cf_scores.index).union(cbf_scores.index)
    hybrid_scores = {}
    for movie_id in all_movies:
        cf_score = cf_scores.get(movie_id, 0)
        cbf_score = cbf_scores.get(movie_id, 0)
        hybrid_scores[movie_id] = w_cf * cf_score + w_cbf * cbf_score
    
    # Get the top recommendations
    hybrid_series = pd.Series(hybrid_scores).sort_values(ascending=False)
    top_movies = hybrid_series.head(top_n).index
    
    # Return the movie titles with scores
    return movies[movies['movieId'].isin(top_movies)].assign(HybridScore=hybrid_series)

# Get recommendations for a user

In [ ]:
try:
    user_id = int(input("Enter the User ID for which you want recommendations: "))
    
    if user_id not in user_item_matrix.index:
        print(f"User ID {user_id} does not exist in the dataset. Please try again.")
    else:
        recommendations = get_hybrid_recommendations(user_id, w_cf=0.5, w_cbf=0.5, top_n=10)
        
        # Display the recommendations
        print(f"\nTop recommended movies for you, {user_id}:")
        print(recommendations)
except ValueError:
    print("Invalid input. Please enter a numeric User ID.")